In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [3]:
import malaya

model = malaya.translation.en_ms.transformer(model = 'bigbird')

/home/ubuntu/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
2022-10-02 23:10:58.989643: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-02 23:10:58.993315: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-02 23:10:58.993334: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-10-02 23:10:5

In [4]:
from unidecode import unidecode

with open('eng_Latn.dev') as fopen:
    eng = fopen.read().split('\n')
    
with open('zsm_Latn.dev') as fopen:
    ms = fopen.read().split('\n')
    
right = [unidecode(s) for s in ms]
left = [unidecode(s) for s in eng]

In [5]:
from tqdm import tqdm

batch_size = 2

results = []
for i in tqdm(range(0, len(left), batch_size)):
    r_greedy = model.greedy_decoder(left[i:i + batch_size])
    results.extend(r_greedy)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 499/499 [13:38<00:00,  1.64s/it]


In [6]:
import re

def bigbird_postprocessing(string):
    return re.sub(r'([!,.?/;\'])\1+', r'\1 ', string).strip()

filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(bigbird_postprocessing(r))
        filtered_right.append(right[no])

In [7]:
refs = [filtered_right]
sys = filtered_left

In [8]:
r = bleu.corpus_score(sys, refs)
r.__dict__

{'name': 'BLEU',
 'score': 39.09071749208737,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '70.5/46.7/32.4/22.9 (BP = 0.989 ratio = 0.989 hyp_len = 21782 ref_len = 22027)',
 'bp': 0.9888152009178546,
 'counts': [15359, 9700, 6411, 4305],
 'totals': [21782, 20785, 19788, 18791],
 'sys_len': 21782,
 'ref_len': 22027,
 'precisions': [70.51234964649711,
  46.668270387298534,
  32.39842328684051,
  22.90990367729232],
 'prec_str': '70.5/46.7/32.4/22.9',
 'ratio': 0.9888772869659963}

In [9]:
chrf.corpus_score(sys, refs)

chrF2++ = 63.96